<a href="https://colab.research.google.com/github/Roger-G/Data-augmentation/blob/master/Data_Replace_by_finetun_twitter_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import package

In [0]:
!pip install transformers
"Fine-tuning BertMasked Model with labeled dataset"
from __future__ import absolute_import, division, print_function
import argparse
import logging
import os
import random
import shutil
import csv
from IPython.display import clear_output
import numpy as np
import torch
import nltk
import pandas as pd
import shutil
nltk.download('averaged_perceptron_tagger')
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords 
from torch.utils.data import DataLoader, RandomSampler, TensorDataset, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import trange
import shutil
from pathlib import Path
PYTORCH_PRETRAINED_BERT_CACHE = Path(os.getenv('PYTORCH_PRETRAINED_BERT_CACHE',
                                               Path.home() / '.pytorch_pretrained_bert'))
# from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from transformers import BertForMaskedLM
from transformers import BertTokenizer
logger = logging.getLogger(__name__)
absFilePath = 'drive/My Drive/Data/Bert_ data augmentation-master/'
clear_output()
train_number = 0 ## this is made for differiate different augmentation data

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Class InputExample

In [0]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.label = label

### Class AugProcessor

In [0]:
class AugProcessor():
  
    """Processor for dataset to be augmented."""
        
    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "train.tsv")), "train") # get_train_examples

    def get_dev_examples(self, data_dir): # get_dev_examples
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "dev.csv")), "dev")

    @ staticmethod
    def get_labels(name): # get_labels
        """add your dataset here"""
        if name in ['toxic', 'rt-polaritydata']:
            return ["0", "1"]
        else:
            return ['positive', 'neutral', 'negative']
          

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid ="%s-%s" % (set_type, i)           
            text = line[1][0]
            label = line[1][-1]
            # print('label', label)
            examples.append(
                InputExample(guid, text, label))
        return examples

    def _read_csv(self, input_file, quotechar='"'):
        """Reads a comma separated value file."""  
        with open(input_file,"r",encoding='UTF-8') as f:
            reader = csv.reader(
                f,
                delimiter=",",
                quotechar=quotechar,
                doublequote=True,
                skipinitialspace=False,
                )

            lines = []
            for index, line in enumerate(reader):
              if index == 10:
                  break
              print(line)
              line = ''.join(line).split('\t')
                # print('line', ''.join(line).split('\t'))
              lines.append((index,line))
            # delete label and sentence
            del lines[0]
            # print('line', lines)
        return lines


### Convert_examples_to_features

In [0]:

def index_value(list_long, list_index, value):
    for i in list_index:
        list_long[i] = value
    return list_long

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, masked_lm_prob):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}
    # print('label_map', label_map)
    features = []
    dupe_factor = 5
    rng = random.Random(123)
    max_predictions_per_seq = 20
    a = examples
    for (ex_index, example) in enumerate(examples):
        # print('example', example)
        tokens_a = tokenizer.tokenize(example.text) # (aaaaa, 1), aaaa is text
        # print('tokens', type(tokens_a))
        tokens_b = None
        if len(tokens_a) > max_seq_length - 2:  # maxlength = [cls]+token_length + [sep]
            tokens_a = tokens_a[:(max_seq_length - 2)]
        # print('label', example.label, len(example.label))
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"] # tokens_a is a token of sentence

        pos = nltk.pos_tag(tokens) 
        index_vb = [ tokens.index(word) for word, tag in pos if tag in ('VBD', 'VB', 'VBG', 'VBN', 'JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNPS', 'NNP' ) 
                    and word not in stopwords.words('english') and word not in ('[CLS]', '[SEP]')]

        s = example.label
    
        try:
            label_id = label_map[s]
        except KeyError:

            label_id = label_map[s.strip()]
        # print('lael_id', label_id)
        segment_ids = [label_id] * len(tokens) # use this label to make label_id and segment_ids # here maybe is the contextual expression
        masked_lm_labels = [0]*max_seq_length

        cand_indexes = []
        for (i, token) in enumerate(tokens):
            if token == "[CLS]" or token == "[SEP]":
                # print('i', i, token)
                continue
            cand_indexes.append(i)
        rng.shuffle(cand_indexes) # 随机替换词为mask，为下面做准备
        # print('cand_indexes', cand_indexes)
        len_cand = len(cand_indexes)
        output_tokens = list(tokens) ## contain '[CLS]' and '[SEP]'
        num_to_predict = min(max_predictions_per_seq, ##  max_predictions_per_seq=20 最多预测20个
                             max(1, int(round(len(tokens) * masked_lm_prob)))) # 最多替换tokens的50%
        # print('num_to_predict', num_to_predict)
        if len(index_vb) > num_to_predict:

            index_vb_new = random.sample(index_vb, num_to_predict)
            masked_lms_pos = index_vb_new
            # print('masked_lms_pos_test', index_vb_new)
            output_tokens = index_value(output_tokens, index_vb_new, '[MASK]')
            masked_lm_labels = index_value(masked_lm_labels, index_vb_new, 1)
            # print('output_tokens', output_tokens)
            
        else:
            output_tokens = index_value(output_tokens, index_vb, '[MASK]')
            masked_lms_pos = index_vb
            # print('< enter else to add po one by one', masked_lms_pos)
            covered_indexes = set(index_vb)
            masked_lm_labels = index_value(masked_lm_labels, index_vb, 1)
            for index in [indx for indx in cand_indexes if indx not in index_vb]: # cand_indexes 是除 [CLS] 和 ['SEP'] 的token 的index集合, 但是已经被shuffle了, sentence
                if len(masked_lms_pos) >= num_to_predict:
                    break
                if index in covered_indexes:
                    continue
                covered_indexes.add(index)

                # mask_pos = []
                masked_token = None # 处理完一个词， should be reset for every token
                # 80% of the time, replace with [MASK]，这个句子的token的80%换成[MASK], 10% keep original, 10% random 

                if rng.random() < 0.8:
                    masked_token = "[MASK]"
                        # output_tokens[index] = masked_token # mask 位置的token
                    masked_lms_pos.append(index)
                        # print('masked_lms_pos', masked_lms_pos)
                    masked_lm_labels[index] = 1 #被mask的位置的原来的token的ids
            
                else:
                    # 10% of the time, keep original
                    # print('rng.random()', rng.random())
                    if rng.random() < 0.5:
                        masked_token = tokens[index]
                    # 10% of the time, replace with random word
                    else:
                        masked_token = tokens[cand_indexes[rng.randint(0, len_cand - 1)]]
                    # print('masked_token', masked_token)
                output_tokens[index] = masked_token
            
        init_ids = tokenizer.convert_tokens_to_ids(tokens) # original tokens
        token_idx = masked_lms_pos
        input_ids = tokenizer.convert_tokens_to_ids(output_tokens) # 80% of the time, replace with [MASK], token的80%probability 换成[MASK], 10% keep original, 10% random 
        input_mask = [1] * len(input_ids)
        padding = [0] * (max_seq_length - len(input_ids))
        init_ids += padding
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(init_ids) == max_seq_length
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join([str(x) for x in tokens]))
            logger.info("init_ids: %s" % " ".join([str(x) for x in init_ids]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("masked_lm_labels: %s" % " ".join([str(x) for x in masked_lm_labels]))

        features.append(
                InputFeatures(init_ids=init_ids, # original tokens ids
                              input_ids=input_ids, # have mask in the sentences and to ids
                              input_mask=input_mask, # padding 0 and no padding 1
                              segment_ids=segment_ids, # if label=='1' then seg_ids = 1* [length of sentence ]
                              masked_lm_labels=masked_lm_labels,
                              token_idx=token_idx)) # 被mask的位置的原来的token的ids
    return features

### Class InputFeatures

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, init_ids, input_ids, input_mask, segment_ids, masked_lm_labels, token_idx):
        self.init_ids = init_ids
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.masked_lm_labels = masked_lm_labels
        self.token_idx = token_idx

### Remove_wordpiece

In [0]:
def remove_wordpiece(str):
    if len(str) > 1:
        for i in range(len(str) - 1, 0, -1):
            if str[i] == '[PAD]':
                str.remove(str[i])
            elif len(str[i]) > 1 and str[i][0] == '#' and str[i][1] == '#':
                str[i - 1] += str[i][2:]
                str.remove(str[i])
    return " ".join(str[1:-1])

In [0]:
def choose_ids(pred_ids, origin_ids): # t=pre, m=origin
  selected_ids = list()
  for i in range(len(pred_ids)):

    if origin_ids[i] == pred_ids[i][0]:
      selected_ids.append(pred_ids[i][1].tolist())
    else:
      selected_ids.append(pred_ids[i][0].tolist())
     
  return selected_ids

## Train

In [0]:
class Train():

  def __init__(self, train_numer, args, num_multi):
    self.train_number = train_number, 
    self.num_multi = num_multi
    self.args = args
  
  
  def path_setting(self, task_name = 'twitter', tech_name = 'Bert_ data augmentation-master'):
    abspath = '/content/drive/My Drive/Data/'
    self.model_path = os.path.join(abspath, tech_name + '/save_model')
    self.dataset_path = os.path.join(abspath, tech_name + '/datasets/{}/'.format(task_name))
    self.save_path = os.path.join(abspath, tech_name + '/aug_data/{}/'.format(task_name))
    
    # return model_path, dataset_path, save_path
  

  @staticmethod
  def load_model(model_name):
    weights_path = os.path.join(self.model_path, model_name)
    model = torch.load(weights_path)
    return model

  def multi(self):
    task_name = self.args.task_name
    self.path_setting()
    self.args.data_dir = self.dataset_path
    print('data_dir', self.args.data_dir)
    self.save_train_path = os.path.join(self.save_path, "{}_times_only_augmentation({})_{}.tsv".format(self.num_multi, self.args.tune_state, train_number))
    print('save_train_path', self.save_train_path)
    self.origin_train_path = os.path.join(self.dataset_path, "train.tsv")
    # print('self.origin_train_path', self.origin_train_path)
    origin = pd.read_csv(self.origin_train_path,  error_bad_lines=False, sep='\t')
    data_len = len(origin)
    origin_copy = origin.copy()
  
    for i, masked_lm_prob in enumerate([0.15, 0.20, 0.30]):
      if i + 1 == self.num_multi:
        break
      '''loading model and do augmentation'''
      save_path = self.Aug(self.train_number, masked_lm_prob, self.args, task_name, self.save_train_path, fine_tune=False)
      new_file = pd.read_csv(save_path)
      origin_copy = pd.concat([origin_copy, new_file[-data_len:]], ignore_index=True)

    origin_copy.to_csv(self.save_path + '{}_times({})_{}.tsv'.format(self.num_multi, self.args.tune_state train_number), sep="\t", index=False)
    pd.concat([new_file, origin], axis=1).to_csv(self.save_train_path, index=False)

  @classmethod
  def Aug(clf, train_number, masked_lm_prob, args, task_name, save_train_path):
    processors = {
        "toxic": AugProcessor,
        'rt-polaritydata': AugProcessor,
        'twitter': AugProcessor
    }

    processor = processors[task_name]()
    label_list = processor.get_labels(task_name)
    ''' change to hugging face version''' 
    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case) 
    train_examples = None
    num_train_steps = None
    train_examples = processor.get_train_examples(args.data_dir)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('device', device)

    if args.tune_state = 'fine_tune':
      cbert_name = args.model_name
      model = clf.load_model(cbert_name)
    else:
      model = BertForMaskedLM.from_pretrained(args.bert_model)
    model.to(device)
    
    # if task_name == 'stsa.fine':
    model.bert.embeddings.token_type_embeddings = torch.nn.Embedding(3, 768)
    model.bert.embeddings.token_type_embeddings.weight.data.normal_(mean=0.0, std=0.02)
    param_optimizer = list(model.named_parameters())
    global_step = 0
    train_features = convert_examples_to_features(
        train_examples, label_list, args.max_seq_length, tokenizer, masked_lm_prob)
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_steps)

    all_init_ids = torch.tensor([f.init_ids for f in train_features], dtype=torch.long)
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
    all_masked_lm_labels = torch.tensor([f.masked_lm_labels for f in train_features], dtype=torch.long)
    train_data = TensorDataset(all_init_ids, all_input_ids, all_input_mask, all_segment_ids, all_masked_lm_labels)
    train_dataloader = DataLoader(train_data,  batch_size=args.train_batch_size)
    MASK_id = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
    
    with open(save_train_path, 'a', encoding='UTF-8') as save_train_file:
      tsv_writer = csv.writer(save_train_file, delimiter=',')
      tsv_writer.writerow(['sentence', 'label'])
      for step, batch in enumerate(train_dataloader):

        model.eval()
        batch = tuple(t.to(device) for t in batch)
        # print('batch', batch)
        init_ids, input_ids, input_mask, segment_ids, all_masked_lm_labels = batch
        masked_idx = []
        for i in all_masked_lm_labels.cpu().numpy():
          s = np.nonzero(i)
          masked_idx.append(s[0].tolist())
        
        predictions = model(input_ids, input_mask, segment_ids)
               
        for ids, idx, preds, seg in zip(init_ids, masked_idx, predictions[0], segment_ids):
          # print('idx',idx)
          # print('original token:',tokenizer.convert_ids_to_tokens(ids[idx].tolist()))
          origin_ids = ids[idx]
          # print('origi ids', origin_ids)
          # idx = masked_idx[int(index)]
          _, indice = torch.topk(torch.softmax(preds[idx], -1), 2)
          selected_ids = choose_ids(indice, origin_ids)
          # print('selected_ids', selected_ids)
          ids[idx] = torch.LongTensor(selected_ids).to(device)   
          pred_str = tokenizer.convert_ids_to_tokens(ids.cpu().numpy())
          
          pred_str = remove_wordpiece(pred_str)
          # print('pred_str', pred_str)
          tsv_writer.writerow([pred_str, seg[0].item()])

    return save_train_path
            

## Parameter

In [0]:

def Parameter(train_number, num_multi):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument("--data_dir", default="datasets", type=str,
                        help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    parser.add_argument("--output_dir", default="aug_data", type=str,
                        help="The output dir for augmented dataset")
    parser.add_argument("--bert_model", default="bert-base-uncased", type=str,
                        help="The path of pretrained bert model.")
    parser.add_argument("--model_name", default="Fune-tune_BERT_twitter", type=str,
                    help="The name of fine-tuned bert model.")
    parser.add_argument("--task_name",default="twitter",type=str,
                        help="The name of the task to train.")
    parser.add_argument("--tune_state",default='unfine', type=str,
                        help='To decide the state of loading model.')
    parser.add_argument("--max_seq_length", default=30, type=int,
                        help="The maximum total input sequence length after WordPiece tokenization. \n"
                             "Sequences longer than this will be truncated, and sequences shorter \n"
                             "than this will be padded.")
    parser.add_argument("--do_lower_case", default=True, action='store_true',
                        help="Set this flag if you are using an uncased model.")
    parser.add_argument("--train_batch_size", default=32, type=int,
                        help="Total batch size for training.")
    parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")

    parser.add_argument('--masked_lm_prob', type=int, default=0.25,
                        help='the probabilty of MASK in original sentence')
    args, unknown = parser.parse_known_args()
#     print(args.data_dir)
    train = Train(train_number, args, num_multi)
    train.multi()
    # Train.Aug()

In [0]:
from sklearn.model_selection import train_test_split
import csv
import os.path as path
df = pd.read_csv('/content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter/train.tsv', sep='\t')
train, test = train_test_split(df, test_size=0.2)

In [0]:
train.reset_index().drop(columns='index').to_csv(path.join('/content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter','train.tsv'), sep='\t')
test.reset_index().drop(columns='index').to_csv(path.join('/content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter','dev.tsv'), sep='\t')

## Execute

In [38]:
if __name__ == "__main__":

  for i in range(2,5):
    print('I want augment {} times'.format(i))
    Parameter(train_number, i) # replace masked_lm_prob words
    
  train_number +=1

  print('\n','Finished!')



I want augment 2 times
data_dir /content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter/
save_train_path /content/drive/My Drive/Data/Bert_ data augmentation-master/aug_data/twitter/2_times_only_augmentation(unfine)_0.tsv
i 0
['sentence\tlabel']
['win the match getin plus tomorrow be very busy day with awareness day and debate gulp debate\tneutral']
['some area of new england could see the first flake of the season tuesday\tneutral']
['nd bad qb definitely tony romo the man who like to share the ball with everyone include the other team\tnegative']
['thailand washington us president barack obama vow wednesday as he visit storm ravage new jersey shore to\tneutral']
['do you all hear what tony romo dress up as for halloween giants quaterback cause that be all he could throw to sunday night\tneutral']
['tim tebow may be availible wow jerry what the heck you wait for\tpositive']
['aga tayo tomorrow ah good night ces love you\tpositive']
['tina fey amy poehler be host t

KeyboardInterrupt: ignored

In [0]:
torch.tensor([i for i in range(100)], dtype=torch.long)

In [0]:
class text():
  d = 1
  b = 3

  def __init__(self):
    self.d = d
    self.b = b
  def test(self):
    print(self.d)


In [0]:
text = text()

In [0]:
text.test()